In [1]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import parse_qs, urlparse
import pandas as pd
import time
import random
from datetime import date, timedelta

# Get the number of pages to scrap
start_url = "https://finviz.com/screener.ashx?v=131&f=sh_avgvol_o100,sh_float_o5,sh_price_o1,sh_relvol_o1,sh_short_o15,ta_rsi_ob60,ta_sma20_pa&ft=4&o=shortinterestshare"
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(start_url, headers=headers)
time.sleep(random.randint(10, 15))

soup = BeautifulSoup(response.text, 'html.parser')
pagination = soup.find_all("a", class_="screener-pages")
last_page = max([int(link.text) for link in pagination if link.text.isdigit()])+1

# Placeholder URL
base_url = "https://finviz.com/screener.ashx?v=131&f=sh_avgvol_o100,sh_float_o5,sh_price_o1,sh_relvol_o1,sh_short_o15,ta_rsi_ob60,ta_sma20_pa&ft=4&o=shortinterestshare&r={}"
headers = {'User-Agent': 'Mozilla/5.0'}

volume_data = []
ticker_data = []
short_float = []
short_ratio = []

page_number = 1

while page_number < last_page:
    url = base_url.format((page_number - 1) * 20 + 1)  # r=1, r=21, r=41,...
    response = requests.get(url, headers=headers)
    time.sleep(random.randint(10, 15))
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find table with float data
    td_list = soup.find_all('td', align='right', height='10')
    print(f"Processing page {page_number}...")
    
    if not td_list:
        break  # Stop if no data found on this page

    # For each row,
    for i in range(8, len(td_list), 14): #3, 8, 9, 13
        td = td_list[i]
        a_tag = td.find('a')

        # Get Float data from the current page
        if a_tag and a_tag.text:
            raw_value = a_tag.text.strip()
            short_float.append(raw_value)

        # Get ticker name from the current page
        if a_tag and a_tag.has_attr('href'):
            href = a_tag['href']
            parsed_url = urlparse(href)
            query_params = parse_qs(parsed_url.query)
            ticker = query_params.get('t', [None])[0]
            
            if ticker:
                ticker_data.append(ticker)

    # Get Short data from the current page
    for i in range(9, len(td_list), 14): #3, 8, 9, 13
        td = td_list[i]
        a_tag = td.find('a')

        # Get Float data from the current page
        if a_tag and a_tag.text:
            raw_value = a_tag.text.strip()
            short_ratio.append(raw_value)

    # Get Short data from the current page
    for i in range(13, len(td_list), 14): #3, 8, 9, 13
        td = td_list[i]
        a_tag = td.find('a')

        # Get Float data from the current page
        if a_tag and a_tag.text:
            raw_value = a_tag.text.strip()
            volume_data.append(raw_value)

    page_number += 1

# present as dataframe, export to csv
df = pd.DataFrame({
    'Ticker': ticker_data,
    'Volume': volume_data,
    'Short Float': short_float,
    'Short Ratio': short_ratio
})


today = date.today() - timedelta(days=1)
filename_date = today.strftime("%y%m%d")
df.to_csv(f'short_float_15p {filename_date}.csv', encoding='utf-8', index=False, header=True)

Processing page 1...


Processing page 2...
